<a href="https://colab.research.google.com/github/PriyankaVerma98/Neural_networks_TensorFlow/blob/master/video6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2
import numpy as np
import imutils
import glob
import os

In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize   

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=aedaa5afc04b8c5a69e0890d1036496e4ce26c3dc2f25a60505cbf60b1436ab7
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [0]:
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
gpu = GPUs[0]

def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print('GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB'.format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

Gen RAM Free: 12.8 GB  I Proc size: 173.7 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


Connect to Google Drive to load required files

In [0]:
directory = '/content/drive/My Drive/Colab Notebooks/'

In [0]:
import sys
sys.path.append(directory)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)





**Generate New Captions**

In [0]:
from keras.preprocessing.text import Tokenizer
from pickle import dump

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# load training dataset (6K)
filename = directory + 'Flickr8k_text/Flickr_8k.trainImages.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions(directory + 'descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
# save the tokenizer
dump(tokenizer, open(directory + 'tokenizer1.pkl', 'wb'))

Using TensorFlow backend.


Dataset: 6000
Descriptions: train=6000


In [0]:
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn't guaranteed
gpu = GPUs[0]

printm()

Gen RAM Free: 12.6 GB  I Proc size: 379.9 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


In [0]:
from pickle import load
from numpy import argmax
from keras.preprocessing.sequence import pad_sequences
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.models import load_model
import numpy as np
import cv2

#OPTIMISED function 

# extract features from each photo in the directory
def extract_features(filename, is_file = 1):
	# load the model
	model = VGG16()
	# re-structure the model
	model.layers.pop()
	model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
	# load the photo
	if is_file==1:
		# image = load_img(filename, target_size=(224, 224))
		image = load_img(filename, target_size=(224, 224))
		# convert the image pixels to a numpy array
		image = img_to_array(image)
	else:
		image = np.resize(filename, (224,224,3))
	# # convert the image pixels to a numpy array
	# image = img_to_array(image)
	# reshape data for the model
	image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
	# prepare the image for the VGG model
	image = preprocess_input(image)
	# get features
	feature = model.predict(image, verbose=0)
	return feature

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text


In [0]:
# load the tokenizer
tokenizer = load(open(directory + 'tokenizer.pkl', 'rb'))
# pre-define the max sequence length (from training)
max_length = 34
# load the model
model = load_model(directory + 'model_20.h5')


In [0]:
GPUs = GPU.getGPUs()
gpu = GPUs[0]
printm()

Gen RAM Free: 11.9 GB  I Proc size: 1.4 GB
GPU RAM Free: 15671MB | Used: 609MB | Util   4% | Total 16280MB


In [0]:
# pathOut = r"/content/drive/My Drive/Colab Notebooks/"
# count = 0
# success = True
# total = []

# while success:
#     success,image = cap.read()
#     if count%40 == 0 :
#         cv2.imwrite(pathOut + '2cook3-%d.jpg'%count,image)
#     count+=2
# printm()

In [0]:
video_capture = cv2.VideoCapture(directory + "check2.mp4")
# Check success
if not video_capture.isOpened():
    raise Exception("Could not open video device")
# Set properties. Each returns === True on success (i.e. correct resolution)
video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, 224)
video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 224)
fps = video_capture.get(cv2.CAP_PROP_FPS)



In [0]:
print(fps)

23.976023976023978


In [0]:
total = [ ]
for i in range(0 , 250 , 60) :
  photo = extract_features(directory + "2cook3-%d.jpg"%i)
  description = generate_desc(model, tokenizer, photo, max_length)
  print(description)
  # im = cv2.imread(directory + "acheck2-%d.jpg"%i)
  # cv2.putText(im, description, (100,500) , cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3, cv2.LINE_AA)
  # cv2.imwrite(directory + '1result%d.jpg'%i,im)
  total.append(description)

startseq man in red shirt is standing on the sidewalk endseq
startseq man in red shirt is standing in front of building endseq
startseq two girls are playing with recently funny endseq
startseq man in red shirt is standing in front of building endseq
startseq man in red shirt is standing in front of building endseq


In [0]:
# # total = [ ]
# # for i in { 6450 ,11760 , 13500, 21900, 32040, 40050 , 35520 } :
# #   photo = extract_features(directory + "acheck2-%d.jpg"%i)
# #   description = generate_desc(model, tokenizer, photo, max_length)
# #   print(description)
# #   im = cv2.imread(directory + "acheck2-%d.jpg"%i)
# #   cv2.putText(im, description, (100,500) , cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3, cv2.LINE_AA)
# #   cv2.imwrite(directory + '1result%d.jpg'%i,im)
#   total.append(description)

In [0]:
# # Read picture. ret === True on success
# success = True 
# count= 0 
# pathOut = r"/content/drive/My Drive/Colab Notebooks/"

# while success:
#     success,image = video_capture.read()
#     if count%3000 == 0 :
#         cv2.imwrite(pathOut + 'acheck2-%d.jpg'%count,image)
#     count+= 3000
#     GPUs = GPU.getGPUs()
#     gpu = GPUs[0]
#     printm()

# Close device
# video_capture.release()

In [0]:
# photo = extract_features(directory + "example.jpg")
# description = generate_desc(model, tokenizer, photo, max_length)
# print(description)
# total.append(description)

In [0]:
# im = cv2.imread(directory + "1result6450.jpg")
# cv2.putText(im, "Hello Piii", (100,500) , cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3, cv2.LINE_AA)
# cv2.imwrite(directory + '1.jpg',im)
